In [72]:
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, threshold=50, linewidth=75)

In [73]:
bikes_numpy = np.loadtxt(
    "../data/p1ch4/bike-sharing-dataset/hour-fixed.csv", 
    dtype=np.float32, 
    delimiter=",", 
    skiprows=1, # skip head row(names of columns)
    converters={1: lambda x: float(x[8:10])}) # <1> first columns string - index 8 and 9 indicates day
    # thus day in string format converted in float
bikes = torch.from_numpy(bikes_numpy)
bikes

tensor([[1.0000e+00, 1.0000e+00,  ..., 1.3000e+01, 1.6000e+01],
        [2.0000e+00, 1.0000e+00,  ..., 3.2000e+01, 4.0000e+01],
        ...,
        [1.7378e+04, 3.1000e+01,  ..., 4.8000e+01, 6.1000e+01],
        [1.7379e+04, 3.1000e+01,  ..., 3.7000e+01, 4.9000e+01]])

In [74]:
# column information
import csv
column_list = next(csv.reader(open("../data/p1ch4/bike-sharing-dataset/hour-fixed.csv"), delimiter=','))
column_list

['instant',
 'dteday',
 'season',
 'yr',
 'mnth',
 'hr',
 'holiday',
 'weekday',
 'workingday',
 'weathersit',
 'temp',
 'atemp',
 'hum',
 'windspeed',
 'casual',
 'registered',
 'cnt']

In [75]:
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))

In [76]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1]) # split data in 24 hrs 
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

In [77]:
daily_bikes = daily_bikes.transpose(1, 2) # N(day samples) x C(columns) x L(length 24hrs)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

In [78]:
first_day = bikes[:24].long()
weather_onehot = torch.zeros(first_day.shape[0], 4) # make 0 tensor with 24 x 4
first_day[:, 9] # extract weather column

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2,
        2, 2])

In [79]:
weather_onehot.scatter_(
    dim=1, 
    index=first_day[:,9].unsqueeze(1) - 1, # weather 1-4, but index should be 0-3
    value=1.0)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        ...,
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [80]:
torch.cat((bikes[:24], weather_onehot), 1)[:1] # concatenate

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,
          6.0000,  0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,
          3.0000, 13.0000, 16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [83]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4, daily_bikes.shape[2])
daily_weather_onehot.shape
#daily_bikes[:,:,9].unsqueeze(2).shape


torch.Size([730, 4, 24])

In [85]:
daily_weather_onehot.scatter_(
    1, daily_bikes[:,9,:].long().unsqueeze(1) - 1, 1.0)
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [86]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)

In [87]:
daily_bikes[:, 9, :] = (daily_bikes[:, 9, :] - 1.0) / 3.0

In [88]:
temp = daily_bikes[:, 10, :]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - temp_min)
                         / (temp_max - temp_min))

In [89]:
temp = daily_bikes[:, 10, :]
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - torch.mean(temp))
                         / torch.std(temp))